# Conectando no google drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Montando o dataset para o treinamento

In [ ]:
import pandas as pd

file_path = 'trn.json'  # O arquivo está na mesma pasta

df = pd.read_json(file_path, lines=True)

df = df[['title', 'content']]
df = df[df['content'].str.len() > 50]

In [ ]:
df = df.rename(columns={'title': 'input', 'content': 'output'})

df['instruction'] = 'Please provide a detailed description of the following product, including its features and any relevant specifications.'
df = df[['instruction', 'input', 'output']]

# Salvando o arquivo novo
df.to_json('trn_output.json', orient='records', lines=True)

# Instalando dependencias

In [1]:
!pip install pandas
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-u6240rx8/unsloth_35ddddc0b50e45db9539b4d91baf9adb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-u6240rx8/unsloth_35ddddc0b50e45db9539b4d91baf9adb
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 13.4 MB/s eta 0:00:00

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Download do modelo

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [4]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

# Prompt com o modelo não tunado

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Experiments with Soap (True Books: Science Experiments)", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Carregando o modelo com parametros

In [ ]:
perf_model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # podemos mudar aqui pois pode mudar algo
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # altera a intensidade da adaptação do lora
    lora_dropout = 0,
    bias = "none", # consideramos que nossos dados n etão enviesados

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Formatando a entrada de dados e carregando dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token # faz o modelo compreender o fim da sentença
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset


dataset = load_dataset("json", data_files='/content/drive/MyDrive/Colab Notebooks/trn_output.json', split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# Configurando treino do modelo

In [ ]:
trainer = SFTTrainer(
    model = perf_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4, # valor baixo para ele ficar atento a novos dados
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # otimiza o treinamento para usar menos memoria
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to=[]
    ),
)

# Treinando o modelo

In [ ]:
trainer_stats = trainer.train()

# Validando dados com o modelo treinado

In [ ]:
FastLanguageModel.for_inference(perf_model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Beautiful Assassin: A Novel", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = perf_model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
FastLanguageModel.for_inference(perf_model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Please provide a detailed description of the following product, including its features and any relevant specifications.",
        "Experiments with Soap (True Books: Science Experiments)", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Salvando o novo modelo treinado

In [ ]:
perf_model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-junior-2") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-junior-2") # saving tokenizer

# Carregando modelo já treinado


In [11]:
from unsloth import FastLanguageModel

llama_fine_tuning_1, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Colab Notebooks/llama-fine-tuning-junior-2",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(llama_fine_tuning_1)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

## Gerando SEED para comparação

In [7]:
import random
import pandas as pd
from datasets import load_dataset

random.seed(42)
dataset = load_dataset("json", data_files='/content/drive/MyDrive/Colab Notebooks/trn_output.json')

df = pd.DataFrame(dataset['train'])

sampled_items = df.sample(n=30, random_state=42)
sampled_items

Generating train split: 0 examples [00:00, ? examples/s]

,instruction,input,output
41475,Please provide a detailed description of the f...,Economics After the Crisis: Objectives and Mea...,"""A well-researched and profound rethink of mac..."
49837,Please provide a detailed description of the f...,I Am a Promise,"""Every child is full of promise, potential, an..."
72420,Please provide a detailed description of the f...,"Trapped (League of Peoples, Bk. 6)",James Alan Gardner is a graduate of the Clario...
134398,Please provide a detailed description of the f...,Dancing In Cadillac Light,"Gr 5-8-It is 1968 and Jaynell Lambert, an 11-y..."
139560,Please provide a detailed description of the f...,The Man Who Found Time: James Hutton And The D...,In this engaging account of scientific discove...
106363,Please provide a detailed description of the f...,The Cambridge Illustrated Atlas of Warfare: Re...,"""The atlas's elegant design and color illustra..."
142629,Please provide a detailed description of the f...,Daily Life Strategies for Teens,Feeling like chaos rules your world? Ready to ...
125791,Please provide a detailed description of the f...,Surveying (10th Edition),Surveying is essentially the science of measur...
32661,Please provide a detailed description of the f...,Getting Rid of Gout,Bryan Emmerson is at University of Queensland.
27145,Please provide a detailed description of the f...,Over 40 &amp; You're Hired!: Secrets to Landin...,Ryan (60 Seconds &amp; You're Hired!) offers a...


In [8]:
def extract_response(text):
    response_marker = "Response:"
    start_index = text.find(response_marker)

    if start_index == -1:
        return None
    response_text = text[start_index + len(response_marker):]
    response_text = response_text.replace('\n', ' ').replace('<|end_of_text|>', '').strip()

    return response_text


def inference(product):
  alpaca_prompt = """Instruction: Please provide a detailed description of the following product, including its features and any relevant specifications.
    Input: {}
    Response:"""


  inputs = tokenizer([alpaca_prompt.format(product)], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
  result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  return result[0]


sampled_items['result_fine_tuning'] = sampled_items['input'].apply(inference).apply(extract_response)

In [9]:
sampled_items

,instruction,input,output,result_fine_tuning
41475,Please provide a detailed description of the f...,Economics After the Crisis: Objectives and Mea...,"""A well-researched and profound rethink of mac...",This book by Robbins is a collection of his le...
49837,Please provide a detailed description of the f...,I Am a Promise,"""Every child is full of promise, potential, an...",I Am a Promise is a book that promises to help...
72420,Please provide a detailed description of the f...,"Trapped (League of Peoples, Bk. 6)",James Alan Gardner is a graduate of the Clario...,"Trapped (League of Peoples, Bk. 6) is a scienc..."
134398,Please provide a detailed description of the f...,Dancing In Cadillac Light,"Gr 5-8-It is 1968 and Jaynell Lambert, an 11-y...","The Dancing In Cadillac Light is a compact, li..."
139560,Please provide a detailed description of the f...,The Man Who Found Time: James Hutton And The D...,In this engaging account of scientific discove...,James Hutton was a Scottish geologist who is w...
106363,Please provide a detailed description of the f...,The Cambridge Illustrated Atlas of Warfare: Re...,"""The atlas's elegant design and color illustra...",The Cambridge Illustrated Atlas of Warfare: Re...
142629,Please provide a detailed description of the f...,Daily Life Strategies for Teens,Feeling like chaos rules your world? Ready to ...,This book is designed to provide teens with th...
125791,Please provide a detailed description of the f...,Surveying (10th Edition),Surveying is essentially the science of measur...,Surveying (10th Edition) is a comprehensive te...
32661,Please provide a detailed description of the f...,Getting Rid of Gout,Bryan Emmerson is at University of Queensland.,Getting rid of gout is a process that involves...
27145,Please provide a detailed description of the f...,Over 40 &amp; You're Hired!: Secrets to Landin...,Ryan (60 Seconds &amp; You're Hired!) offers a...,Over 40 &amp; You're Hired!: Secrets to Landin...


In [10]:
output_file = "/content/drive/MyDrive/Colab Notebooks/model_original.csv"
sampled_items.to_csv(output_file, index=True)